# Closed loop simulations

In this notebook we solve the control problem at the vehicle level. The algorithm is a centralized predictive controller. The notebook is divided in multiple sections 

- [Vehicle dynamics](#dynamics)
- [Control problem](#control_problem)
    - [Optimality conditions](#optimal_conditions) 
- [Functions at operational level](#compute_control)
- [Connection with symuvia and data storage](#closed_loop)

## System dynamics 

<a id='dynamics'></a>

In addition the dynamics of the system are defined as 

\begin{equation}
\begin{bmatrix}
\dot{x}_i\\
\dot{s}_i\\
\dot{v}_i\\
\end{bmatrix}=
\begin{bmatrix}
v_i\\
v_{i-1}-v_i\\
u_i
\end{bmatrix}
\end{equation}

<a id='control_problem'></a>
## Control problem 

In this case the solution to the optimal control problem corresponds to the solution of the optimal problem:

\begin{equation} 
\begin{aligned}
    \underset{\mathbf u}{\min}\ J\big(\mathbf{x},\mathbf{u},t|\mathbf{x}(0)\big) =& \underset{\mathbf u}{\min}& &\int^{T_p}_0 \mathcal{L}(\mathbf x (\tau), \mathbf u(\tau),\tau)\quad d\tau + \mathcal{G}(\mathbf x(T_p),T_p)\\
    & \text{s.t} & & \mathbf x(0)= \mathbf x_0\\
    & & & \dot{\mathbf x} = \mathbf f(\mathbf x, \mathbf u)
\end{aligned}
\end{equation}

for a determined cost function $\mathcal{L}(\mathbf x, \mathbf u)$. 

## Cost function

In this case the cost function is defined as follows:

\begin{equation}
\begin{aligned}
\mathcal{L}(s_i,v_i) = C_1 \big(s_i-v_i\cdot t_h+s_0\big)^2 + C_2\big(v_i-v_{i-1}\big)^2+C_3u^2_i
\end{aligned}
\end{equation}

where $s_i$ represents the spacing of $i$-vehicle, $v_i$ its speed and $a_i$ the vehicle acceleration, $u_i$ is the external acceleration to be applied by the control. The parameters $t_h, s_0$ represent the time headway, and constant space reference.

<a id='optimal_conditions'></a>
#### Optimality conditions

The solution of the optimal control problem requires the implementation of the Hamiltonian given by
\begin{equation}
    \mathcal{H}(\mathbf x,\mathbf u) = \mathcal{L}(\mathbf x,\mathbf u) +\lambda^T\mathbf{f}(\mathbf x,\mathbf u)
\end{equation}


For the given boundary value problem (BVP), two simultaneous initial value problems (IVP) are solved simultaneously. The solution that satisfies 
\begin{equation}
    \mathbf u^\star = \min \mathcal{H}(\mathbf x, \mathbf u, \lambda), \quad \text{s.t}\ \mathbf u \in \mathcal{U}, x \in \mathcal{X}
\end{equation}
when the second differential equation is satisfied by the by the co-state dynamic equation 
\begin{equation}
\begin{aligned}
-\frac{d}{dt}\lambda =& \frac{\partial\mathcal H}{\partial \mathbf x} = \frac{\partial \mathcal L}{\partial \mathcal x}+\lambda \cdot \frac{\partial \mathbf f}{\mathbf x}\\
\lambda(T_p)&=\frac{\partial \mathcal G}{\partial x}(\mathbf x(T_p))
\end{aligned}
\end{equation}

#### Optimality conditions (2nd order dynamics)

The hamiltonian can be written this time as:

\begin{equation}
\begin{aligned}
\mathcal{H}(s_i,v_i,u_i) =& C_1 \left(s_i-(v_i\cdot t_g+s_0)\right)^2 + C_2\big(v_{i-1}-v_{i}\big)^2+C_3u^2_i+ \lambda_s(v_{i-1}-v_{i})+\lambda_v u_i
\end{aligned}
\end{equation}

The conditions lead to the following conditions in the case of the following conditions:

\begin{equation}
    \begin{aligned}
    -\frac{\partial \lambda_s}{\partial t} & = \frac{\partial \mathcal H}{\partial s_i} = 2C_1\left(s_i -(v_it_g+s_0)\right) & \rightarrow &\dot{\lambda}_s = -2C_1\left(s_i -(v_it_g+s_0)\right) \\
    -\frac{\partial \lambda_v}{\partial t} & = \frac{\partial  \mathcal H}{\partial v_i} = -2C_1\left(s_i -(v_it_g+s_0)\right)t_g-2C_2(v_{i-1}-v_{i})-\lambda_s & \rightarrow &\dot{\lambda}_v = 2C_1\left(s_i -(v_it_g+s_0)\right)t_g + 2C_2(v_{i-1}-v_{i})+\lambda_s
    \end{aligned}
\end{equation}

In addition, 

\begin{equation}
\begin{aligned}
    u^\star_i &= \frac{\partial \mathcal H}{\partial u_i} = 2C_3u_i + \lambda_v = 0\\
    u^\star_i &= -\frac{\lambda_v}{2c_3}
\end{aligned}
\end{equation}

Package importing  

In [18]:
import os 

from sqlalchemy import create_engine, MetaData
from sqlalchemy import Table, Column, String, Integer, Float 
from sqlalchemy import insert, delete, select, case, and_

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Bokeh
from bokeh.plotting import figure, show
from bokeh.sampledata.iris import flowers
from bokeh.io import output_notebook
from bokeh.palettes import Viridis
output_notebook() 


import ipywidgets as widgets
from IPython.display import display

Loading BokehJS ...

<a id='table_creation'></a>
#### Creation of tables

In this case table `headway` and `traj` should be already filled according to notebooks [01-Open-loop.ipynb](01-Open-loop.ipynb) and [02-Tactical-strategy.ipynb](02-Tactical-strategy.ipynb)

In [19]:
dir_path = os.getcwd()

engine_path = ('..','Output','SymOut.sqlite')
engine_name = os.path.join(os.path.sep,*engine_path)
engine_full_name = os.path.join(dir_path,*engine_path)
engine_call = 'sqlite://'+engine_name
engine = create_engine(engine_call)
metadata = MetaData()

if os.path.isfile(engine_full_name):
    try:
        ltbstr = 'Loaded table in: '
        connection = engine.connect()    
        traj = Table('traj', metadata, autoload=True, autoload_with=engine)
        headway = Table('headway', metadata, autoload=True, autoload_with=engine)        
        closed = Table('closed', metadata, autoload=True, autoload_with=engine)
        control = Table('control', metadata, autoload=True, autoload_with=engine)
        stmt = delete(closed)          
        results = connection.execute(stmt)
        stmt = delete(control)
        results = connection.execute(stmt)        
    except:
        ltbstr = 'Created table in: '
        closed = Table('closed', metadata,
                 Column('ti', Float()),
                 Column('id', Integer()),
                 Column('type', String(3)),
                 Column('tron', String(10)),
                 Column('voie', Integer()),
                 Column('dst', Float()),
                 Column('abs', Float()),
                 Column('vit', Float()),
                 Column('ldr', Integer()),
                 Column('spc', Float()),
                 Column('vld', Float()))
        control= Table('control', metadata,
                 Column('ti', Float()),
                 Column('id', Integer()),
                 Column('type', String(3)),
                 Column('tron', String(10)),
                 Column('voie', Integer()),
                 Column('ctr', Float()),
                 Column('nit', Integer())) 
        metadata.create_all(engine)
        connection = engine.connect()
    finally: 
        print(ltbstr, engine)
                

Loaded table in:  Engine(sqlite:///../Output/SymOut.sqlite)


#### Functions for operational level 

The following auxiliary functions are created in order to solve the problem in the function [compute_control](#compute_control)

- `reversedEnumerate` Consider a list of `np.array` and flips the array along the 0 axis
- `initial_setup_mpc` Initializes variables for the controller 
- `find_idx_ldr` Finds the leader of vehicles at time t. 
- `format_reference` converts the reference information form a db into a matrix. 

In [20]:
def reversedEnumerate(*args):
    """ Inverse enumeration iterator"""
    revArg = [np.flip(x, axis = 0) for x in args]
    return zip(range(len(args[0])-1, -1, -1), *revArg)

def find_idx_ldr(results):
#     """ From dbQuery finds idx or leader for CAVs"""
    # Frozen network (A-priori)
    ldrl = [dveh_ldr[x[1]] for x in results if x[2]=='CAV']
    idx_ldr = [dveh_idx[x] for x in ldrl]
    
    return idx_ldr, ldrl  

def initial_setup_mpc(results, h_ref):
    """ Initialize variables for controller
    """
    
    TGref = format_reference(h_ref)  
    h = TGref.shape[0] 
    
    n_CAV = len([ty[2] for ty in results if ty[2]=='CAV'])
    dCAVu = [h, n_CAV]
    # print(f'Dimensions control: {dCAVu}')

    Sref = np.zeros(dCAVu)

    S = np.zeros(dCAVu)
    V = np.zeros(dCAVu)
    DV = np.zeros(dCAVu)
    Lv = np.zeros(dCAVu)
    Ls = np.zeros(dCAVu)
    return (Sref, TGref, S, V, DV, Ls, Lv)

def format_reference(h_ref):
    """ Convert query from a reference into a 
        numpy array
    """
    vehids = set([x[1] for x in h_ref])
    
    # Rearrange 
    refDf = pd.DataFrame(h_ref, columns = ['ti','id','gapt'])
    # Pivot to pass vehicles as columns
    refMat = pd.pivot_table(refDf, index='ti', columns='id')['gapt']
    refMat = refMat.as_matrix()
    
    return refMat 

<a id='compute_control'></a>
#### Control definition 

The function computes the optimal control for a set of vehicles

In [21]:

def compute_control(results, h_ref, u_lead):
              
    Sref, Tgref, S, V, DV, Ls, Lv = initial_setup_mpc(results, h_ref)
    
    # Static leadership
    ldr_pos, ldr_list = find_idx_ldr(results)
    
    S0 = [s[9] for s in results if s[2]=='CAV']
    V0 = [v[7] for v in results if v[2]=='CAV']
    DV0 = [dv[10]-dv[7] for dv in results if dv[2]=='CAV']
    U_ext = Lv
    #U_ext[:,0] = u_lead # Head acceleration (external)

    
    # Initialize global variables 
    S[0] = S0
    V[0] = V0
    DV[0] = DV0    
    h = len(S)
    n = 0
    n_prev = 0
    
    # Parameters 
    C1 = 0.1
    C2 = 1
    C3 = 0.5
    ALPHA = 0.01
    1
    EPS  = 0.1
    error = 100
    
    
    bSuccess = 2 
    N = 100001 # number of iterations
    step = iter(range(N))
    
    while (error > EPS) and (bSuccess>0):
        try: 
            next(step)
            U_star = -Lv/(2*C3)
            U_star = np.clip(U_star, U_MIN, U_MAX) 
            
            DU = U_star[:,ldr_pos]-U_star[:] + U_ext 

            # Forward evolution
            for i,u_s,du in zip(range(h), U_star, DU):        
                if i<len(S)-1:
                    DV[i+1] = DV[i]+ DT * du
                    S[i+1] = S[i] + DT * DV[i]
                    V[i+1] = V[i]+ DT * u_s
        

            Sref = V * Tgref + 1/KC
            
            # Forward plots 
            # plot_forward(Sref, Tgref, S, V, DV, U_star)
            
            ls = np.zeros(Ls.shape) 
            lv = np.zeros(Lv.shape)


            # Backward evolution
            for i, s, v, dv, tg in reversedEnumerate(S, V, DV, Tgref):            
                if i>0:
                    sref = v * tg + 1/KC
                    lv[i-1] = lv[i] + DT * (-2 * C1 * (s-sref) * tg - C2 * dv - ls[i])
                    ls[i-1] = ls[i] + DT * (2 * C1 * (s-sref))
        

            # Update 
            Ls = (1 - ALPHA) * Ls + ALPHA * ls
            Lv = (1 - ALPHA) * Lv + ALPHA * lv      
            
            # Backwards plots 
            # plot_backwards(ls, lv, Ls, Lv)

            error = np.linalg.norm(Ls - ls) + np.linalg.norm(Lv-lv)
            # print(f'Iteration: {n}, Error: {error}')
            

            # Routine for changing convergence parameter

            if error > 10e5:
                raise AssertionError('Algorithm does not converge ')
            if n >= 500:
                ALPHA = max(ALPHA - 0.01, 0.01)
                #print(f'Reaching {n} iterations: Reducing alpha: {ALPHA}')  
                #print(f'Error before update {error}')
                if n > 10000:
                    raise AssertionError(
                        'Maximum iterations reached by the algorithm')
                n_prev = n + n_prev
                n = 0
            if error <= EPS:
                bSuccess = 0
                
            n += 1
    
        except StopIteration:
            print('Stop by iteration')
            print('Last simulation step at iteration: {}'.format(n+n_prev))
            bSuccess = 0
    
    n = n + n_prev
    return (S, V, DV, U_star, DU, n)

def determine_lane_change(CAVabsP):
    """ Returns the tuple (tron, voie) for a 
        CAV vehicle based on positions updates.
    """    
    
    # Works only in current network
    
    CAVtron = []
    CAVvoie = []
    
    for abs_x in CAVabsP:
        if (abs_x <= 0):
            CAVtron.append('In_main')
            CAVvoie.append(1)
        elif (abs_x > 0) and (abs_x <=100.0):
            CAVtron.append('Merge_zone')
            CAVvoie.append(2)            
        else:
            CAVtron.append('Out_main')
            CAVvoie.append(1)
    return CAVtron, CAVvoie

def update_state(S, V, DV, U_star, DU, n, results_closed):
    """ Updates the state and computes closed loop updates
    """

    # NOTE: To be taken into account. Closed loop simulations
    # run without Symuvia. Requires implementation of the connection 
    # NO LANE CHANGE MODEL IMPLENTED FOR HDV 
    
     # Forward evolution 
    DVp = DV[0] + DT * DU[0]
    Sp = S[0] + DT * DV[0]
    Vp = V[0] + DT * U_star[0] 
    
    # t_i, id, type, from (results_closed):
    # Keep the same along the simulation
    CAVti = [x[0]+ DT for x in results_closed if x[2]=='CAV']
    CAVti = [float(np.round(x,1)) for x in CAVti]
    CAVid = [x[1] for x in results_closed if x[2]=='CAV']
    CAVtype = [x[2] for x in results_closed if x[2]=='CAV']

    # Updates

    # Postition query
    CAVdst = [x[5] for x in results_closed if x[2]=='CAV']   
    CAVabs = [x[6] for x in results_closed if x[2]=='CAV']  
    
    # Updates from closed loop
    CAVdstP = [x+ DT * v for (x,v) in zip(CAVdst, V[0])] #or Vp?
    CAVabsP = [x+ DT * v for (x,v) in zip(CAVabs, V[0])] #or Vp?    
    
    
    # Lane change 
    CAVtron, CAVvoie = determine_lane_change(CAVabsP)
    
    # State updates 
    ldr_pos, ldr_list = find_idx_ldr(results_closed)
    CAVvitP = Vp
    CAVldr = ldr_list
    CAVldr = [int(x) for x in ldr_list]
    CAVspcP = Sp
    CAVvldP = DVp + Vp
    
    # Render CAV results for dB
    CAViter = zip(CAVti, CAVid, CAVtype, CAVtron, CAVvoie,
                  CAVdstP, CAVabsP, CAVvitP, CAVldr, 
                  CAVspcP, CAVvldP)
    
    keys = ('ti', 'id', 'type', 'tron', 'voie', 
            'dst', 'abs', 'vit', 'ldr', 
            'spc','vld')
    
    keysU = ('ti', 'id', 'type', 'tron', 'voie',
             'ctr', 'nit')
    
    lVehTrajCL = []
    for i in CAViter:
        lVehTrajCL.append(dict(zip(keys,i)))
    
    
    n_list = [n]*U_star.shape[0]
    
    CAViter = zip(CAVti, CAVid, CAVtype, CAVtron, 
                  CAVvoie, U_star[0], n_list)
    
    lVehUCL = []
    for i in CAViter:
        lVehUCL.append(dict(zip(keysU,i)))
        
    return lVehTrajCL, lVehUCL


def format_open_loop(results):
    """ Aux function 
        To write in the closed loop database 
        results from open loop without treatment
        (No control applied)
        Homogenizes results in terms of content
    """
    
    keys = ('ti', 'id', 'type', 'tron', 'voie', 
            'dst', 'abs', 'vit', 'ldr', 
            'spc','vld')
    keysU = ('ti', 'id', 'type', 'tron', 'voie',
             'ctr', 'nit')
    
    lVehTrajOL = []
    lVehUOL = []
    for i in results:
        lVehTrajOL.append(dict(zip(keys,i)))
        ti, vid, ty, tr, vo, _, _, _, _, _, _ = i
        u_tup = (ti, vid, ty, tr, vo, 0, 0)
        lVehUOL.append(dict(zip(keysU,u_tup)))
        
    return lVehTrajOL,lVehUOL


<a id='closed_loop'></a>
### Launch closed loop simulation 

Closed loop simulations 

In [22]:
progressSim = widgets.FloatProgress(
    value=5,
    min=0,
    max=71.9,
    step=0.1,
    description='Simulating:',
    bar_style='info',
    orientation='horizontal'
)

Parameters 

In [23]:
DT = 0.1 # Sample time 

KC = 0.16 # CAV max density 
KH = 0.0896 # HDV max density
VF = 25.0 # Speed free flow
W = 6.25 # Congestion speed 
E  = 25.0*0.3 # Speed drop for relaxation 

GCAV = 1/(KC*W) # Time headway CAV 
GHDV = 1/(KH*W) # Time headway HDV 
SCAV = VF/(KC*W)+1/KC #  Desired space headway CAV 
SHDV = VF/(KH*W)+1/KH #  Desired space headway HDV

dveh_twy = {'CAV': GCAV, 'HDV': GHDV}
dveh_dwy = {'CAV': 1/KC, 'HDV': 1/KH}

U_MAX = 1.5 # Max. Acceleration
U_MIN = -1.5 # Min. Acceleration

Leader static information. This part freezes the leader so reference of the conrol does not change abruptly 

In [24]:
stmt = select([traj])
results = connection.execute(stmt).fetchall()
column_names = traj.columns.keys()
trajDf = pd.DataFrame(results, columns = column_names)
network_data = trajDf[(trajDf['ti']==20) & (trajDf['type']=='CAV')]
key = network_data.id.values
ldr = network_data.ldr.values
idx = range(len(key))
dveh_ldr = dict(zip(key,ldr))
dveh_idx = dict(zip(key,idx))

Closed loop 

In [25]:
%%time
N = 1200 # Simulation steps 

t = np.round(np.arange(718)/10,2)
t = [str(t_i) for t_i in t]

t_it = iter(t)
step = iter(range(N)) 

# Cleaning closed loop tables 
stmt = delete(closed) 
connection.execute(stmt)
stmt = delete(control) 
connection.execute(stmt)

# Query definitions 
stmtwriteCL = insert(closed) 
stmtwriteUCL = insert(control)

t_ip = 0.0

display(progressSim) 

bSuccess = 2 
while bSuccess>0:
    try: 
        t_i = next(t_it)
        step_i = next(step)
                
        # Prediction horizon
        t_f = float(t_i) + 5.0
        stmt = select([headway]).where(and_(headway.columns.ti>=t_i,
                                            headway.columns.ti<=t_f))
        h_ref = connection.execute(stmt).fetchall() 
     
        
        if step_i>=125:   
            # Closed looop             
            # print(f'Closed loop connected: {t_i, step_i}')
            # print(f'Solved in {n} iterations - Window: {t_i,t_f}')
            stmt = select([closed]).where(and_(closed.columns.ti==t_ip, 
                                               closed.columns.type=='CAV'))
            results = connection.execute(stmt).fetchall() 
            
            S, V, DV, U_star, DU, n = compute_control(results, h_ref, 0) 
            
            lVehTrajCL, lVehU = update_state(S, V, DV, U_star, DU, n, results)
            
        else: 
            # Open loop
            # print(f'Open loop connected: {t_i, step_i}')
            stmt = select([traj]).where(and_(traj.columns.ti==t_i, 
                                             traj.columns.type=='CAV'))
            results = connection.execute(stmt).fetchall()            
            
            lVehTrajCL, lVehU = format_open_loop(results)
        
        if lVehTrajCL:            
            connection.execute(stmtwriteCL,lVehTrajCL)
            connection.execute(stmtwriteUCL,lVehU)        
        
        t_ip = t_i
        progressSim.value = t_i  
    except StopIteration:
        print('Stop by iteration')
        print('Last simulation step at time: {}'.format(t_i))
        bSuccess = 0
        

FloatProgress(value=5.0, bar_style='info', description='Simulating:', max=71.9)

Stop by iteration
Last simulation step at time: 71.7
CPU times: user 2min 26s, sys: 2.31 s, total: 2min 28s
Wall time: 2min 36s


In [ ]:
refdBDf.to_sql(name='headway', con = engine, if_exists='replace', index=False)